# Customer Lifetime  Value (CLV) Prediction

In [23]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import lifetimes

from datetime import datetime
from dateutil.relativedelta import relativedelta

import os
from google.cloud import bigquery

### Import Data from BigQuery

In [24]:
# Import order data
gcr_project_id = os.getenv('GCR_CLV_PROJECT_ID')
QUERY  = f"""
WITH order_values AS (
    SELECT 
      order_id,
      SUM(sale_price) as order_value
    FROM `{gcr_project_id}.the_look_ecommerce_constant.order_items`
    GROUP BY order_id
    ORDER BY order_id
)
SELECT
  orders.order_id,
  orders.user_id,
  users.first_name,
  users.last_name,
  users.email,
  orders.created_at,
  orders.status,
  order_values.order_value
FROM `{gcr_project_id}.the_look_ecommerce_constant.orders` AS orders
    LEFT JOIN `{gcr_project_id}.the_look_ecommerce_constant.users` AS users ON orders.user_id = users.id
    LEFT JOIN order_values on orders.order_id = order_values.order_id
ORDER BY orders.order_id;
"""

client = bigquery.Client()

df = client.query_and_wait(QUERY).to_dataframe()

/Users/axel/ecommerce-project/env/lib/python3.11/site-packages/google/cloud/bigquery/table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


### Import data from local files

! NOTE: BigQuery completely regenerates the dataset everyday thus it is recommended to import the data from a locally saved version in order to ensure it remains constant throughout the model creation process.

In [2]:
df_orders = pd.read_csv('data/orders.csv')
df_order_items = pd.read_csv('data/order_items.csv')
df_users = pd.read_csv('data/users.csv')

order_values = df_order_items.groupby('order_id', as_index=False).agg({'sale_price':'sum'}).sort_values(by='order_id').rename(columns={'sale_price':'order_value'})
df = pd.merge(df_orders[['order_id','user_id','created_at','status']],
              df_users[['id','first_name','last_name','email']], how='left', left_on='user_id', right_on='id')
df = pd.merge(df,
              order_values[['order_id','order_value']], how='left', on='order_id').sort_values(by='order_id').reset_index(drop=True)
df['created_at'] = df.created_at.apply(lambda x : datetime.strptime(x.split(" ")[0], "%Y-%m-%d"))
df

NameError: name 'pd' is not defined

### Formatting Data - Calculate Frequency, Recency, Customer Age, and Customer Monetary Value.

In [25]:
# Reformat 'created_at' column the exclude time of day
from datetime import datetime
df2 = df.copy()
df2['created_at'] = df2.created_at.apply(lambda x : x.date())
df2.head()

,order_id,user_id,first_name,last_name,email,created_at,status,order_value
0,1,2,Erica,Wright,ericawright@example.com,2024-06-15,Shipped,89.000000
1,2,3,Robert,Moore,robertmoore@example.net,2024-06-08,Cancelled,54.990002
2,3,3,Robert,Moore,robertmoore@example.net,2023-08-23,Complete,108.359999
3,4,4,Rachel,Wright,rachelwright@example.org,2023-12-17,Shipped,14.990000
4,5,5,Melissa,Buck,melissabuck@example.com,2024-04-08,Processing,172.000000


In [26]:
# Get Customer Summary Data : Frequency, Recency, Monetary Value
df_rfm  = lifetimes.utils.summary_data_from_transaction_data(df2, 'user_id', 'created_at',
                                                            freq='D', include_first_transaction = False)
df_rfm['monetary_value'] = df2.groupby('user_id')[['order_value']].mean()
df_rfm.head()

,frequency,recency,T,monetary_value
user_id,,,,
2,0.0,0.0,221.0,89.000
3,1.0,290.0,518.0,81.675
4,0.0,0.0,402.0,14.990
5,1.0,172.0,289.0,148.655
6,0.0,0.0,1565.0,152.000


### Defining Utility Functions for Evaluating CLV Prediction Model Performance

This section contains the definition of the following functions:

* `get_train_test_rfm` - get training and testing datasets for evaluating a predictive model. The training dataset contains RFM data for shoppers whose first purchase was made during the training period. The testing dataset contains the true RFM and equity data of these shoppers during the testing period.
* `get_pred_equity` - predict the future equity of users whose data was used to fit the model during the prediction period.
* `eval_predictions` - calculate metrics to evaluate the models predictions.

In [27]:
# Dataset creation functions
def get_train_test_rfm(df : pd.DataFrame, 
                        train_period_start : datetime.date,
                        train_period_end : datetime.date,
                        prediction_period_duration : int=12):
    """Get RFM summary dataframes for the training & testing periods.
    
    Args:
        df - dataframe containing order data
        train_period_start - start of training period
        train_period_end - end of training period
        prediction_period_duration - duration of the prediction period in months
        
    Returns:
        Tuple[df_train_rfm, df_test_rfm, df_all_rfm]
    """
            
    # Create copy of dataset & reformat 'created_at' column
    df1 = df.copy()
    df1['created_at'] = df1.created_at.apply(lambda x : x.date())

    # Calculate end of prediction period and assert ensure that it ends before the maximum order date
    prediction_period_end = train_period_end + relativedelta(months=prediction_period_duration)
    assert df1.created_at.max() > prediction_period_end, f"Prediction period ends in the future - we do not have the data to evaluate these predictions : try reducing the prediction period duration or choosing an earlier training period end date"

    # Filter out users who were 'alive' before training period started
    first_order_dates = df1.groupby('user_id')['created_at'].min()
    valid_users = first_order_dates[(first_order_dates >= train_period_start) & (first_order_dates <= train_period_end)].index
    df1 = df1[df1['user_id'].isin(valid_users)]

    # Select data from training & testing periods
    df_train = df1[(df1.created_at <= train_period_end)]
    df_test = df1[(df1.created_at > train_period_end) & (df1.created_at <= prediction_period_end)]

    # Get RFM summary data from training period
    df_train_rfm  = lifetimes.utils.summary_data_from_transaction_data(df_train, 'user_id', 'created_at',
                                                                           freq='D', include_first_transaction = False)
    df_train_rfm = pd.merge(df_train_rfm, df_train.groupby('user_id')['order_value'].agg(['mean', 'sum']), 
                            how='left', on='user_id').rename(columns={'mean' : 'monetary_value', 'sum' : 'revenue'})
    
    # Get RFM summary data from testing period
    df_test_rfm = lifetimes.utils.summary_data_from_transaction_data(df_test, 'user_id', 'created_at',
                                                                     freq='D', include_first_transaction = True)
    df_test_rfm = pd.merge(df_test_rfm, df_test.groupby('user_id')['order_value'].agg('sum'), 
                           how='left', on='user_id').rename(columns={'frequency' : 'true_purchases', 'order_value':'true_equity'})
    
    # Combine training & testing RFM data
    df_all_rfm = pd.merge(df_train_rfm.rename(columns={'frequency':'train_frequency', 'recency':'train_recency', 'T':'train_T'}), 
                          df_test_rfm.rename(columns={'recency':'test_recency', 'T':'test_T'}), how='left', left_index=True, right_index=True)

    
    return df_train_rfm, df_test_rfm, df_all_rfm

In [28]:
# Prediction functions
def get_pred_equity(model,
                    prediction_period_duration : int=12,
                    discount_rate : float=0.1, freq : str="D"):
    """Predict the equity of training dataset shoppers during the prediction period.
    
    Args:
        model - prediction model that has already been fitted with the training dataset
        prediction_period_duration - duration of the prediction period in months
        
    Returns:
        pred_equity - dataframe containing predicted equity for each shoppper
    """
    
    pred_equity = model.predict_clv(prediction_period_duration, discount_rate, freq).rename(columns={'clv':'pred_equity'})

    return pred_equity

In [ ]:
# Evaluation functions
def eval_predictions(pred_equity : pd.Series,
                     true_equity : pd.Series):
    """Calculates metrics to evaluate equity predictions"""

    error_description = (true_equity.fillna(0)-pred_equity).describe().to_frame().T[['count', 'mean', '50%', 'min', 'max']].rename({'50%':'median'}, axis=1)
    prop_underpredicted = round(sum(true_equity.fillna(0) > pred_equity)/error_description['count'],3)
    prop_overpredicted = round(sum(pred_equity > true_equity.fillna(0))/error_description['count'],3)
    true_vs_pred_diff = round(sum(true_equity.fillna(0) - pred_equity),2)

    df_metrics = pd.concat([error_description,
                            pd.DataFrame({'prop_underpredicted' : prop_underpredicted,
                                          'prop_overpredicted' : prop_overpredicted,
                                          'true_vs_pred_diff' : true_vs_pred_diff})], axis=1)
    return df_metrics

### Gamma-Gamma CLV Prediction Model

In [30]:
# Create class for Gamma-Gamma prediction model
class PredictorGGF:
    def __init__(self, df_summary):
        self.model_name = "GGF"
        self.df_summary = df_summary
        self.correlation = self.df_summary[self.df_summary.frequency != 0][['monetary_value', 'frequency']].corr().values[0,1]

        return print(f"Correlation between shopper frequency & monetary value is : {float(self.correlation):.5f}.")

    def fit_bgf(self, penalty_coef : float=0.01):

        self.bgf = lifetimes.BetaGeoFitter(penalty_coef)
        self.bgf.fit(self.df_summary['frequency'],
                    self.df_summary['recency'],
                    self.df_summary['T'])

        print(f"Beta-Gamma model successfully fitted")
        return self.bgf.summary

    def fit_ggf(self, penalty_coef : float=0.01):
        assert self.correlation < 0.1, f"Correlation between frequency and monetary value for returning customers is {self.correlation} - this is quite high and may cause poor predictions"

        self.ggf = lifetimes.GammaGammaFitter(penalty_coef)
        self.ggf.fit(self.df_summary[self.df_summary.frequency != 0]['frequency'],
                     self.df_summary[self.df_summary.frequency != 0]['monetary_value'])

        print(f"Gamma-Gamma model successfully fitted")
        if float(self.ggf.params_['q']) < 1:
            print("Outliers in the data are causing the 'q' parameter for the Gamma-Gamma model to be < 1 therefore model predictions will fail.\nFix this by either removing outliers until you get 'q' > 1, or use raw monetary values to model CLV.")

        return self.ggf.summary
    
    def predict_clv(self, time : int=12, discount_rate : float=0.1, freq : str="D"):
        """Predict Customer Lifetime Value
        Args:
            time (float, optional) – the lifetime expected for the user in months. Default: 12
            discount_rate (float, optional) – the monthly adjusted discount rate. Default: 0.01
            freq (string, optional) – {“D”, “H”, “M”, “W”} for day, hour, month, week. This represents what unit of time your T is measure in.

        Returns:
            Series – Series object with customer ids as index and the estimated customer lifetime values as values
        """

        # Predict customer lifetime value
        clv_preds_df = self.ggf.customer_lifetime_value(
                            self.bgf,
                            self.df_summary['frequency'],
                            self.df_summary['recency'],
                            self.df_summary['T'],
                            self.df_summary['monetary_value'],
                            time=time,
                            discount_rate=discount_rate,
                            freq=freq
                        ).to_frame()
        
        return clv_preds_df

In [51]:
# Define training period end dates to test on
testing_periods = [(datetime(2024,6,30).date(),6), (datetime(2024,3,31).date(),9), (datetime(2023,12,31).date(),12),
                     (datetime(2023,6,30).date(),6), (datetime(2023,3,31).date(),9), (datetime(2022,12,31).date(),12)]

results_data = {'train_period_end' : [], 'test_period_duration' : [], 'train_period_duration' : [], 'count_pred_upcoming_shoppers' : [], 
                'pred_error_mean' : [], 'pred_error_median' : [], 'pred_error_min' : [], 'pred_error_max' : [], 'prop_underpredicted' : [],'prop_overpredicted' : [],'pred_error_sum' : [], '%_test_period_total_revenue' : []}

train_period_start = datetime(2020,1,1).date()

for end_date, test_duration in testing_periods:
    results_data['train_period_end'].append(end_date)
    results_data['test_period_duration'].append(test_duration)
    results_data['train_period_duration'].append(round((end_date-train_period_start).days/30.5,1))

    df_train, df_test, df_all = get_train_test_rfm(df, train_period_start = train_period_start, train_period_end = end_date, 
                                                   prediction_period_duration=test_duration)

    # Fit GGF model to training data
    ggf_model_exp1 = PredictorGGF(df_train)
    bgf_summary = ggf_model_exp1.fit_bgf()
    ggf_summary = ggf_model_exp1.fit_ggf()

    # Predict shopper equity during prediction period
    pred_equity = get_pred_equity(ggf_model_exp1, prediction_period_duration=8)

    # Add predicted equity to testing period data
    df_all = pd.merge(df_all, pred_equity, how='left', left_on='user_id', right_index=True)

    # Get results metrics of cases where shoppers were less than 90 days old at end of training period & predicted equity is greater than 0
    results = eval_predictions(df_all[(df_all.pred_equity > 0) & (df_all.train_T < 90)]['pred_equity'],
                               df_all[(df_all.pred_equity > 0) & (df_all.train_T < 90)]['true_equity'])
    
    results_data['count_pred_upcoming_shoppers'].append(results['count'].values[0])
    results_data['pred_error_mean'].append(round(results['mean'].values[0],2))
    results_data['pred_error_median'].append(round(results['median'].values[0],2))
    results_data['pred_error_min'].append(round(results['min'].values[0],2))
    results_data['pred_error_max'].append(round(results['max'].values[0],2))
    results_data['prop_underpredicted'].append(round(results['prop_underpredicted'].values[0],2))
    results_data['prop_overpredicted'].append(round(results['prop_overpredicted'].values[0],2))
    results_data['pred_error_sum'].append(round(results['true_vs_pred_diff'].values[0],2))
    results_data['%_test_period_total_revenue'].append(round(results['true_vs_pred_diff'].values[0]*100/df_test['true_equity'].sum(),2))

df_results = pd.DataFrame(data=results_data)
df_results

Correlation between shopper frequency & monetary value is : -0.00425.
Beta-Gamma model successfully fitted
Gamma-Gamma model successfully fitted
Outliers in the data are causing the 'q' parameter for the Gamma-Gamma model to be < 1 therefore model predictions will fail.
Fix this by either removing outliers until you get 'q' > 1, or use raw monetary values to model CLV.
Correlation between shopper frequency & monetary value is : -0.00551.
Beta-Gamma model successfully fitted
Gamma-Gamma model successfully fitted
Outliers in the data are causing the 'q' parameter for the Gamma-Gamma model to be < 1 therefore model predictions will fail.
Fix this by either removing outliers until you get 'q' > 1, or use raw monetary values to model CLV.
Correlation between shopper frequency & monetary value is : -0.00354.
Beta-Gamma model successfully fitted
Gamma-Gamma model successfully fitted
Outliers in the data are causing the 'q' parameter for the Gamma-Gamma model to be < 1 therefore model predicti

,train_period_end,test_period_duration,train_period_duration,count_pred_upcoming_shoppers,pred_error_mean,pred_error_median,pred_error_min,pred_error_max,prop_underpredicted,prop_overpredicted,pred_error_sum,%_test_period_total_revenue
0,2024-06-30,6,53.8,427.0,28.27,-9.76,-105.40,668.86,0.34,0.66,12071.35,1.36
1,2024-03-31,9,50.9,300.0,29.77,-9.91,-131.91,496.91,0.38,0.62,8930.12,0.84
2,2023-12-31,12,47.9,257.0,45.51,-8.19,-121.80,850.05,0.44,0.56,11695.36,1.02
3,2023-06-30,6,41.8,152.0,2.77,-14.15,-130.33,737.72,0.16,0.84,421.78,0.11
4,2023-03-31,9,38.9,136.0,26.02,-9.40,-68.82,669.34,0.35,0.65,3539.10,0.72
5,2022-12-31,12,35.9,126.0,13.42,-9.59,-71.77,304.42,0.31,0.69,1690.70,0.31


### Raw Monetary Value CLV Prediction Model

In [15]:
# Create class for Raw Monetary Value prediction model
import lifetimes.utils
class PredictorRawMonetary:
    def __init__(self, df_summary):
        self.model_name = "Raw Monetary Value"
        self.df_summary = df_summary
        self.correlation = self.df_summary[self.df_summary.frequency != 0][['monetary_value', 'frequency']].corr().values[0,1]

        return print(f"Correlation between shopper frequency & monetary value is : {float(self.correlation):.5f}.")

    def fit_bgf(self, penalty_coef : float=0.01):

        self.bgf = lifetimes.BetaGeoFitter(penalty_coef)
        self.bgf.fit(self.df_summary['frequency'],
                    self.df_summary['recency'],
                    self.df_summary['T'])

        print(f"Beta-Gamma model successfully fitted")
        return self.bgf.summary
    
    def predict_clv(self, time : int=12, discount_rate : float=0.1, freq : str="D"):
        """Predict Customer Lifetime Value
        Args:
            time (float, optional) – the lifetime expected for the user in months. Default: 12
            discount_rate (float, optional) – the monthly adjusted discount rate. Default: 0.01
            freq (string, optional) – {“D”, “H”, “M”, “W”} for day, hour, month, week. This represents what unit of time your T is measure in.

        Returns:
            Series – Series object with customer ids as index and the estimated customer lifetime values as values
        """

        # Predict customer lifetime value
        clv_preds_df = lifetimes.utils._customer_lifetime_value(
                            self.bgf,
                            self.df_summary['frequency'],
                            self.df_summary['recency'],
                            self.df_summary['T'],
                            self.df_summary['monetary_value'],
                            time=time,
                            discount_rate=discount_rate,
                            freq=freq
                        ).to_frame()
        
        return clv_preds_df

In [16]:
rmv_model = PredictorRawMonetary(df_rfm)
penalty_val = 0.01
bgf_summary = rmv_model.fit_bgf(penalty_coef=penalty_val)
clv_preds = rmv_model.predict_clv()
clv_preds

Correlation between shopper frequency & monetary value is : -0.00532.
Beta-Gamma model successfully fitted


,clv
user_id,
2,14.937287
3,16.589154
4,1.990773
5,41.951372
6,8.657667
...,...
99996,15.316959
99997,25.083725
99998,81.014759


In [17]:
# Experiment 1 : Training Period = 1/1/2022 - 30/4/2024, Testing Period = 1/5/2024 - 31/12/2024
df_train, df_test, df_all = get_train_test_rfm(df, train_period_start=datetime(2022,1,1).date(), train_period_end=datetime(2024,4,30).date(), prediction_period_duration=8)

# Fit GGF model to training data
rmv_model_exp1 = PredictorRawMonetary(df_train)
penalty_val = 0.01
bgf_summary = rmv_model_exp1.fit_bgf(penalty_coef=penalty_val)

# Predict shopper equity during prediction period
pred_equity = get_pred_equity(rmv_model_exp1, prediction_period_duration=8)

# Add predicted equity to testing period data
df_all = pd.merge(df_all, pred_equity, how='left', left_on='user_id', right_index=True)
df_all

Correlation between shopper frequency & monetary value is : 0.00847.
Beta-Gamma model successfully fitted


,train_frequency,train_recency,train_T,monetary_value,revenue,true_purchases,test_recency,test_T,true_equity,pred_equity
user_id,,,,,,,,,,
3,0.0,0.0,251.0,108.359999,108.359999,1.0,0.0,205.0,54.990002,11.602000
4,0.0,0.0,135.0,14.990000,14.990000,NaN,NaN,NaN,NaN,1.887717
5,0.0,0.0,22.0,172.000000,172.000000,1.0,0.0,94.0,125.310000,26.164122
7,1.0,466.0,646.0,52.315001,104.630001,NaN,NaN,NaN,NaN,6.571584
11,0.0,0.0,599.0,51.000000,51.000000,NaN,NaN,NaN,NaN,3.771303
...,...,...,...,...,...,...,...,...,...,...
99983,1.0,115.0,343.0,46.975000,93.950001,NaN,NaN,NaN,NaN,7.259763
99984,0.0,0.0,678.0,165.980000,165.980000,NaN,NaN,NaN,NaN,11.469650
99986,1.0,334.0,495.0,52.000000,104.000000,NaN,NaN,NaN,NaN,7.492092


In [18]:
# Assess overall predictions
eval_predictions(df_all['pred_equity'], df_all['true_equity'])

# Asses cases where prediced equity is > 50 - Model is overpredicting the true equity in these cases
eval_predictions(df_all[df_all.pred_equity > 50]['pred_equity'], df_all[df_all.pred_equity > 50]['true_equity'])

# Asses cases where prediced equity is > 50 - Model is overpredicting the true equity in these cases
eval_predictions(df_all[df_all.pred_equity > 100]['pred_equity'], df_all[df_all.pred_equity > 100]['true_equity'])

# Asses cases where shoppers were less than 90 days old at end of training period - Model is still overpredicting in these cases, but to a less extent
eval_predictions(df_all[(df_all.train_T < 90)]['pred_equity'], df_all[(df_all.train_T < 90)]['true_equity'])

,count,mean,median,min,max,prop_underpredicted,prop_overpredicted,true_vs_pred_diff
0,5954.0,19.154958,-5.534903,-175.865646,1393.142051,0.256802,0.743198,NaN
